## PICMUS beamforming with usbmd toolbox
The PICMUS dataset is one of the most widely used ultrasound datasets in the research community. The dataset is available at https://www.creatis.insa-lyon.fr/Challenge/IEEE_IUS_2016/.
In this tutorial we show you how to use the dataset with the usbmd toolbox.

First make sure you have installed the usbmd toolbox using our [installation instructions](../../README.md). Next, we need to prepare the PICMUS dataset for use with the toolbox. After download, convert the dataset to the usbmd format using the following command:
```bash
python usbmd/data/convert/picmus.py --source <path_to_downloaded_dataset> --output <path_to_output_dir>
```

Also make sure to then correctly setup your data paths by running
 ```bash
 python usbmd/datapaths.py
 ```
All paths in the repository are relative to the root of the repository you set using the previous step.

Then add the (relative) location of your converted PICMUS dataset to `./configs/config_picmus_rf.yaml`. In this tutorial, we assume the following relative path:

```yaml
# ./configs/config_picmus_rf.yaml
data:
  dataset_folder: "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf"
```


In [1]:
# Change the working directory to the root of the project by moving up in the
# directory tree until the file '.gitignore' is found.
import os

while not os.path.exists('.gitignore'):
    os.chdir('..')

# Set to True to run the notebook quickly for testing purposes This is done
# automatically in 'tests/test_run_notebooks.py'
quick_mode = True

In [2]:
# injected parameters
quick_mode = False

In [3]:
from usbmd.ui import DataLoaderUI
from usbmd.setup_usbmd import setup
from usbmd.ops import Pipeline, Downsample, Demodulate, Normalize, EnvelopeDetect, LogCompress, DelayAndSum, TOFCorrection

2024-10-08 15:01:22.706241: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 15:01:22.767501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 15:01:22.767548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 15:01:22.769211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 15:01:22.777773: I tensorflow/core/platform/cpu_feature_guar

### PICMUS

### Experiments - resolution distortion

#### RF beamforming
Let's load the RF data and beamform it. First have a look at the most easy way to do it using our `DataloaderUI` class and setup config functions.

In [4]:
# Loads config, sets data paths and initializes gpu if available
config = setup(config_path="./configs/config_picmus_rf.yaml")

# Override config settings
config.plot.save = False
config.plot.headless = True

# data paths
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf"
config.data.file_path = "contrast_speckle_simu_dataset_rf.hdf5"
config.plot.headless = False # you can set to False to see the plots

usbmd: Using config file: ./configs/config_picmus_rf.yaml
usbmd: Git branch and commit: feature/ops_beamformer=4a4c0f572bfe68775df01006b17043eb7702e3d1
usbmd: WARNING output path `output` does not exist, please update your users.yaml file.


-------------------GPU settings-------------------
     memory
GPU        
0      9640
1     11008
2     11008
3     11008
4     11008
5     11008
6     11008
7     11008
Selected GPU 1 with Free Memory: 11008.00 MiB


/ultrasound-toolbox/usbmd/datapaths.py:101: UnknownHostnameWarning: Unknown hostname bmdserver3 for user devcontainer15 and no default data_root found.
  warnings.warn(


Hiding GPUs [0, 2, 3, 4, 5, 6, 7] from the system.
--------------------------------------------------


In [5]:
if not quick_mode:
    ui = DataLoaderUI(config)
    image = ui.run(plot=True)

usbmd: Dataset was validated on January 18, 2024
usbmd: Remove /mnt/z/Ultrasound-BMd/data/USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf/validated.flag if you want to redo validation.


ImportError: Cannot load backend 'QtAgg' which requires the 'qt' interactive framework, as 'headless' is currently running

#### IQ beamforming
We can also load the IQ data and beamform it.

No need to reload a new config (although you can use `./configs/config_picmus_iq.yaml`).
We can simply set n_ch to 2 and change the data file path

In [ ]:
config.scan.n_ch = 2
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_iq"
config.data.file_path = "contrast_speckle_simu_dataset_iq.hdf5"

In [ ]:
if not quick_mode:
    ui = DataLoaderUI(config)
    image = ui.run(plot=True)

#### IQ beamforming, but reading RF data
For this, we need a more manual approach. Let's first load the RF data, but now in `raw_data` format. After that we can manually convert it to IQ data and beamform it. We can now either again use the `Process` class, or we can construct our own custom processing `Pipeline`.

In [ ]:
# data paths

config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/contrast_speckle/contrast_speckle_simu_dataset_rf"
config.data.file_path = "contrast_speckle_simu_dataset_rf.hdf5"
config.scan.n_ch = 1
config.scan.downsample = 4

if not quick_mode:
    ui = DataLoaderUI(config)
    # >> load data
    raw_data_rf = ui.get_data()
    # select transmits to use
    raw_data_rf = raw_data_rf[ui.scan.selected_transmits]

    # >> convert RF to IQ
    print(f"RF data has shape {raw_data_rf.shape}")

    # >> process data
    # Let's make a pipeline to process the data
    demodulate_func = Demodulate(fs=ui.scan.fs, fc=ui.scan.fc, ops=config.ml_library)

    raw_data_rf = demodulate_func.prepare_tensor(raw_data_rf)
    raw_data_iq = demodulate_func.process(raw_data_rf)

    print(f"IQ data has shape {raw_data_iq.shape}")

    # now we have IQ data, the number of channels is 2
    ui.scan.n_ch = 2
    # >> beamform data
    pipeline = Pipeline([
        TOFCorrection(),  # raw_data -> aligned_data
        DelayAndSum(),  # aligned_data -> beamformed_data
        EnvelopeDetect(),  # beamformed_data -> envelope_data
        Downsample(),  # envelope_data -> envelope_data
        Normalize(),  # envelope_data -> envelope_data
        LogCompress(),  # envelope_data -> image
    ],
        ops=config.ml_library,
        batch_dim=False,
    )
    pipeline.set_params(config, ui.scan, ui.probe)
    pipeline.initialize()

    image = pipeline.process(raw_data_iq)
    ui.dtype = "image"
    ui.data = image

    # >> plot image
    ui.plot(image)

### Simulations - resolution distortion
For fun, let's look at another PICMUS dataset as an example. This is a simulated dataset, generating a bunch of scatterers. Often we can use it to look at the resolution of our ultrasound pipeline.

#### RF beamforming

In [ ]:
# data paths
config.data.dataset_folder = "USBMD_datasets/PICMUS/database/simulation/resolution_distorsion/resolution_distorsion_simu_dataset_rf"
config.data.file_path = "resolution_distorsion_simu_dataset_rf.hdf5"
config.scan.n_ch = 1

if not quick_mode:
    ui = DataLoaderUI(config)
    image = ui.run()

The quality is not really impressive. But wait, in the config we specified the number of transmits (in this case number of plane wave angels) to be 7. The total in the dataset is 75. Let's see how it looks using all of them. 
This can take a bit longer... Reduce the number of transmits or use cpu instead of gpu if you run into any issues.

In [ ]:
if not quick_mode:
    config.scan.selected_transmits = 30
    ui = DataLoaderUI(config)
    image = ui.run(plot=True)